# Project 2 -- Gossip Style Failure Detectors

## Instructions

Please read carefully:

* Solve the project yourself. No teamwork.
* If you have questions, please post these in the public channel on Slack. The answers may be relevant to others as well. 
* Feel free to import and use any additional Python package you need.
* You are allowed to solve the project using a different programming language. In this case, please send me your full code and instructions how to run it (in this case you may have to use a different socket library).
* Your code may be tested on more than 2 nodes. Two nodes are used for the sake of an example.
* In case you experience errors when running your code - read the error carefully. It may happen that the port has not been freed and you just need to wait for a few more seconds to fix the problem.
* Make sure to fill in your `student_name` in the following block below.

In [3]:
student_name = 'David Mihola' # fill with your student name
assert student_name != 'your_student_name', 'Please fill in your student_name before you start.'

You will be sending and receiving messages via sockets. We will use ZeroMQ library, is a  high-performance asynchronous messaging library, aimed at use in distributed or concurrent applications. ZeroMQ simplifies communication handling between distributed nodes. ZeroMQ reference guide: https://zguide.zeromq.org/docs/chapter2/ 


Below you will find an example of generating a pair of communicating nodes, which periodically exchange messages. Both nodes eventually terminate. 

In [4]:
import random
import multiprocessing
import time
import os
import sys
import numpy as np
import socket
import pickle
import signal
import colorama as cm

# constants
RUN_LENGTH = 600 # lenght of the simulation in seconds
STATE_PRINTOUT_PERIOD = 15 # period of node states report
NUMBER_OF_PRINTOUTS = RUN_LENGTH // STATE_PRINTOUT_PERIOD

# play with the number of nodes and their fail/restart probabilities
NODES_FAIL_RESTART_PROBS = [
                               (0.15, 0.02), (0.05, 0.05), (0.02, 0.15), (0.01, 0.1), # arbitrarily chosen probabilities
                               (0.0, 0.0), # always running node
                               (1.0, 1.0)  # on/off node (quite interresting, how this one behaves)
                           ]

HEARTBEATS = "HBs"
SENDER_ID = "s_id"
IP = "127.0.0.1" # localhost
PORT_OFFSET = 5500 

T_MUL_CONST = 0.75 # this seems to work quite nice
T_ADD_CONST = 0

T_GOSSIP = lambda _: (
    3
) # The paper suggests 'T_GOSSIP = xN/B', where: x is the number of bytes per gossip message,
  #                                              N is the number of processes and
  #                                              B is the available bandwith.
  # In this case, on localhost the bandwith is CPU bound - usually in Gb/s, the number of processes is expected to be low and
  # the number of bytes send by each process is also low, although increasing with number of processes, as the heartbeat vector increases. 
  # But still in this setting the gossip time using this equation would be very small, therfore a minimum gossip time of 3 s is 
  # introduced, as the paper also suggests.

T_FAIL = lambda numnodes, multiplicative_constant = 1, additive_constant = 0: (
    T_GOSSIP(numnodes) * numnodes * np.log(numnodes) * multiplicative_constant + additive_constant
) # From the graphs and explanations in the paper, I undetstood that 'T_FAIL' should be multiplied linearithmically by the number of nodes. 
  # By adding aditive and multiplicative constans, the 'P_misstake' can be further decreased/increased.

T_CLEANUP = lambda numnodes, multiplicative_constant = 1, additive_constant = 0: (
    2 * T_FAIL(numnodes, multiplicative_constant, additive_constant)
) # 2 * 'T_FAIL'

# node states in the heartbeat vector
UNKNOWN_NODE = -3
FAILED_NODE = -2
CLEANED_NODE = -1
ACTIVATED_NODE = 0

COLORS = True # Set to 'False' to print without colors, otherwise printing without colors is only to non-terminal devices.
TEXT_COLORS = [cm.Fore.LIGHTBLUE_EX, cm.Fore.LIGHTGREEN_EX, cm.Fore.MAGENTA, cm.Fore.LIGHTYELLOW_EX, cm.Fore.LIGHTRED_EX, cm.Fore.CYAN, 
               cm.Fore.YELLOW, cm.Fore.GREEN, cm.Fore.BLUE, cm.Fore.LIGHTMAGENTA_EX, cm.Fore.LIGHTCYAN_EX, cm.Fore.RED]
TEXT_COLOR_LEN = len(TEXT_COLORS)

def color(node_id):
    if COLORS:
        return f"{TEXT_COLORS[node_id % TEXT_COLOR_LEN]}\33[1m"
    else:
        return ""

def bg_fail():
    if COLORS:
        return f"{cm.Back.RED}{cm.Fore.WHITE}"
    else:
        return ""

def bg_success():
    if COLORS:
        return f"{cm.Back.GREEN}{cm.Fore.WHITE}"
    else:
        return ""

def bg_warning():
    if COLORS:
        return f"{cm.Back.YELLOW}{cm.Fore.WHITE}"
    else:
        return ""

def bg_unknown():
    if COLORS:
        return f"{cm.Back.BLACK}{cm.Fore.WHITE}"
    else:
        return ""

def reset():
    if COLORS:
        return f"{cm.Style.RESET_ALL}"
    else:
        return ""

class Node(multiprocessing.Process): # each node is a process
    def __init__(self, id, number_of_nodes, print_lock, fail_prob, restart_prob):
        super(Node, self).__init__()
        self.id = id
        self.number_of_nodes = number_of_nodes
        self.print_lock = print_lock
        self.fail_prob = fail_prob
        self.restart_prob = restart_prob
    
    def init(self): # other initialization only after fork() to prevent copying memory
        self.ones = np.ones(self.number_of_nodes)
        self.t_cleanup = T_CLEANUP(N, T_MUL_CONST, T_ADD_CONST) * self.ones
        self.t_fail = T_FAIL(N, T_MUL_CONST, T_ADD_CONST) * self.ones
        self.t_gossip = T_GOSSIP(N)
        self.heartbeats = np.zeros(self.number_of_nodes) * UNKNOWN_NODE # A node does not know the state of other nodes at the beginning.
        self.heartbeats[self.id] = ACTIVATED_NODE                       # But sets itsel to active.
        self.last_heartbeats = np.zeros(self.number_of_nodes)  * UNKNOWN_NODE
        self.last_heartbeats[self.id] = ACTIVATED_NODE
        
        # As the paper suggests "In gossip protocols, a member forwards new information to randomly chosen members.", the communication
        # should be unicast, not some kind of broadcast as in the original code
        self.gossip_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # internet - UDP, transmission of gossip messages does not have to be reliable
        self.receiver_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.receiver_socket.bind((IP, PORT_OFFSET + self.id)) # node listens on this socket
        # sockets will be closed by the OS

        self.fail_timestamps = None
        self.cleanup_timestamps = None
        self.failed = False
    
    def timestamps_reset(self):
        current_time = time.time() # initiate timestamps to current time
        self.fail_timestamps = self.ones * current_time
        self.cleanup_timestamps = self.ones * current_time

    def stop(self):
        self.terminate()
        self.join()
        
    def print_state(self, *_):
        with self.print_lock:
            print(f"{color(self.id)}Node {self.id}{reset()} is {f'{bg_fail()}not working{reset()}.' if self.failed else f'{bg_success()}working{reset()}, its perception of the network is:'}{reset()}", file=sys.stderr)
            if not self.failed:
                for i, heartbeat in enumerate(self.heartbeats):
                    if i != self.id:
                        print(f"   {color(i)}Node {i}{reset()} is {f'{bg_warning()}failed' if heartbeat == FAILED_NODE else f'{bg_fail()}cleaned' if heartbeat == CLEANED_NODE else f'{bg_unknown()}unknown' if heartbeat == UNKNOWN_NODE else f'{bg_success()}running'}{reset()}.", file=sys.stderr)
    
    def verbose_heartbeats(self):
        heartbeats = "[ "
        for heartbeat in self.heartbeats:
            if heartbeat == FAILED_NODE:
                heartbeats += "failed "
            elif heartbeat == CLEANED_NODE:
                heartbeats += "cleaned "
            elif heartbeat == UNKNOWN_NODE:
                heartbeats += "unknown "
            else:
                heartbeats += f"{int(heartbeat)} "
        heartbeats += "]"
        return heartbeats
    
    def register_signal_handlers(self):
        signal.signal(signal.SIGUSR1, self.print_state) # signal handler to print the state of a node
        signal.siginterrupt(signal.SIGUSR1, False)

    def run(self):
        self.register_signal_handlers()
        self.init() # initialize necessary variables

        # Randomly waiting before nodes start to gossip as described in the paper: "In practice, the protocols are not run in rounds. 
        # Each member gossips at regular intervals, but the intervals are not synchronized."
        sleep_time = random.uniform(0.0, N)
        with self.print_lock:
            print(f"{color(self.id)}Node {self.id}{reset()} is waiting for {sleep_time} s befor sending first gossip messages...", file=sys.stderr)
        time.sleep(sleep_time)

        self.timestamps_reset()

        while True:
            if self.failed:
                if random.random() <= self.restart_prob: # Process can be restarted with a given probability.
                    self.failed = False
                    self.timestamps_reset()
                    for i in range(N):
                        self.heartbeats[i] = UNKNOWN_NODE # Node does not know the state of other nodes after restarting.
                        self.last_heartbeats[i] = UNKNOWN_NODE

                    self.heartbeats[self.id] = ACTIVATED_NODE # Sets its heartbeat to active again.
                    self.last_heartbeats[self.id] = ACTIVATED_NODE
                    with self.print_lock:
                        print(f"{color(self.id)}Node {self.id}{reset()} {bg_success()}restarted{reset()}.", file=sys.stderr)
            else:
                self.send_gossip()
                if random.random() <= self.fail_prob: # Process can fail with a given probability.
                    self.failed = True
                    with self.print_lock:
                        print(f"{color(self.id)}Node {self.id}{reset()} {bg_fail()}failed{reset()}.", file=sys.stderr)

            try: # Waiting on a single socket is used instead of polling multiple sockets, which was used in the previous code. 
                 # Combined waiting up to T_GOSSIP, then new gossip message must be sent.
                sleep_start = time.time() # timestamp of the start of the waiting
                current_time = sleep_start
                while sleep_start + self.t_gossip > current_time: # waited less than for T_GOSSIP
                    self.receiver_socket.settimeout(sleep_start + self.t_gossip - current_time) # set the timeout to wait exactly for T_GOSSIP to be fullfilled
                    message = self.receiver_socket.recvfrom(65507) # expecting the gossip message is never larger than maximum UDP payload size
                    message = pickle.loads(message[0]) # deserilize
                    if not self.failed: # Message is processed only if the node is running.
                        self.process_gossip(message) # Parses the gossip message and updates heartbeats.
                    current_time = time.time()

            except: # socket timed out, waiting for T_GOSSIP was fullfiled
                pass
    
    def send_gossip(self):
        heartbeats_numpy = np.array(self.heartbeats)
        heartbeats_numpy[self.id] = CLEANED_NODE # Mark self as cleaned to exclude the possibility of sending messages to itself.
        heartbeats_numpy = np.where((heartbeats_numpy != FAILED_NODE) & (heartbeats_numpy != CLEANED_NODE))[0] # exclude failed and cleaned processes

        if heartbeats_numpy.shape[0]: # only send message if there is any listener
            listener_id = heartbeats_numpy[random.randint(0, heartbeats_numpy.shape[0] - 1)] # select randomly another node

            with self.print_lock:
                print(f"{color(self.id)}Node {self.id}{reset()} gossips to {color(listener_id)}node {listener_id}{reset()}.", file=sys.stdout)

            self.heartbeats[self.id] += 1 # increase its heartbeat
            status = { SENDER_ID: int(self.id), HEARTBEATS: self.heartbeats }
            self.gossip_socket.sendto(
                pickle.dumps(status), (IP, PORT_OFFSET + listener_id)
            ) # serilize and send a message to one specific node - unicast

    def process_gossip(self, message):
        current_time = time.time()
        recieved_heartbeats = message[HEARTBEATS]
        updated_heartbeats_mask = recieved_heartbeats > self.last_heartbeats  # mask, where recieved hearbeats are higher than current

        self.fail_timestamps *= ~updated_heartbeats_mask # mask out changed timestamps
        self.fail_timestamps += current_time * updated_heartbeats_mask # set masked timestamps to current time

        self.cleanup_timestamps *= ~updated_heartbeats_mask # mask out changed timestamps
        self.cleanup_timestamps += current_time * updated_heartbeats_mask # set masked timestamps to current time

        updated_heartbeats = np.max(
            np.stack((self.last_heartbeats, recieved_heartbeats)
        ), axis=0).astype(np.float64) # merge the heartbeat lists and adopt the maximum value for each member

        cleaned_processes_mask = self.cleanup_timestamps < (current_time - self.t_cleanup) # mask, where heartbeats weren't updated for T_CLEANUP
        updated_heartbeats *= ~cleaned_processes_mask # mask out cleaned processes
        updated_heartbeats += self.ones * CLEANED_NODE * cleaned_processes_mask # set the cleaned processes as cleaned
        self.last_heartbeats = updated_heartbeats.copy() # keep a copy, where failed processes remain with current heartbeat value

        failed_processes_mask = (self.fail_timestamps < (current_time - self.t_fail)) & ~cleaned_processes_mask # mask, where heartbeats weren't updated for T_FAIL and aren't cleaned
        updated_heartbeats *= ~failed_processes_mask # mask out failed processes
        updated_heartbeats += self.ones * FAILED_NODE * failed_processes_mask # set fail processes as failed, so they are not resend

        self.heartbeats = updated_heartbeats # update the heartbeats, which are going to be sent in the next gossip iteration

        with self.print_lock:
            print(f"{color(self.id)}Node {self.id}{reset()} received gossip from {color(message[SENDER_ID])}node {message[SENDER_ID]}{reset()}, neighborship list: {color(self.id)}{self.verbose_heartbeats()}{reset()}", file=sys.stdout)

def run_network(network, print_lock):
    for node in network: # start node processes
        node.start()
    
    for i in range(1, NUMBER_OF_PRINTOUTS + 1): # run the simulation for a given time
        time.sleep(STATE_PRINTOUT_PERIOD)
        with print_lock:
            print(f"\nState of the network at time {i * STATE_PRINTOUT_PERIOD}:", file=sys.stderr) 
        for node in network:
            os.kill(node.pid, signal.SIGUSR1)

    for node in network: # terminate and join node processes
        node.stop()

if __name__ == "__main__":
    if not sys.stdout.isatty(): # When the output of the script is not a terminal, print without colors.
        COLORS = False

    print(f"{reset()}", end="", file=sys.stdout)
    N = len(NODES_FAIL_RESTART_PROBS)
    print(f"T_gossip: {T_GOSSIP(N)} s, T_fail: {T_FAIL(N, T_MUL_CONST, T_ADD_CONST)} s, T_cleanup: {T_CLEANUP(N, T_MUL_CONST, T_ADD_CONST)} s", file=sys.stderr)

    print_lock = multiprocessing.Lock() # Process safe mutex to ensure processes, that don't print over each other.
    network = [Node(id, N, print_lock, fail_prob, restart_prob)
                  for id, (fail_prob, restart_prob) in enumerate(NODES_FAIL_RESTART_PROBS)]
    try:
        run_network(network, print_lock)
    except KeyboardInterrupt: # ensure all processes terminate
        with print_lock:
            print("\nTerminating...\n", file=sys.stderr)
        for node in network:
            node.stop()


T_gossip: 3 s, T_fail: 24.188752834578743 s, T_cleanup: 48.377505669157486 s
Node 0 is waiting for 3.747487646598334 s befor sending first gossip messages...
Node 1 is waiting for 1.753268911718806 s befor sending first gossip messages...
Node 2 is waiting for 4.140564143594352 s befor sending first gossip messages...
Node 3 is waiting for 0.24305675419612593 s befor sending first gossip messages...
Node 4 is waiting for 4.9603408660014825 s befor sending first gossip messages...
Node 5 is waiting for 5.321942949692031 s befor sending first gossip messages...


Node 3 gossips to node 0.
Node 1 gossips to node 0.
Node 3 gossips to node 1.
Node 1 received gossip from node 3, neighborship list: [ 0 0 0 2 0 0 ]
Node 0 gossips to node 2.
Node 0 received gossip from node 3, neighborship list: [ 0 0 0 1 0 0 ]
Node 0 received gossip from node 1, neighborship list: [ 0 1 0 1 0 0 ]
Node 2 gossips to node 0.
Node 0 received gossip from node 2, neighborship list: [ 0 1 1 1 0 0 ]
Node 2 received gossip from node 0, neighborship list: [ 1 0 0 0 0 0 ]
Node 1 gossips to node 4.
Node 4 gossips to node 0.
Node 4 received gossip from node 1, neighborship list: [ 0 1 0 2 0 0 ]
Node 0 received gossip from node 4, neighborship list: [ 0 1 1 1 1 0 ]
Node 5 gossips to node 1.


Node 5 failed.


Node 1 received gossip from node 5, neighborship list: [ 0 0 0 2 0 1 ]
Node 3 gossips to node 5.
Node 0 gossips to node 1.
Node 1 received gossip from node 0, neighborship list: [ 1 1 1 2 1 1 ]
Node 2 gossips to node 3.
Node 3 received gossip from node 2, neighborship list: [ 1 0 1 0 0 0 ]
Node 1 gossips to node 2.
Node 2 received gossip from node 1, neighborship list: [ 1 2 1 2 1 1 ]
Node 4 gossips to node 2.
Node 2 received gossip from node 4, neighborship list: [ 1 2 1 2 1 1 ]


Node 5 restarted.


...

Node 5 failed.


Node 2 gossips to node 0.
Node 4 gossips to node 1.


Node 5 restarted.


Node 2 gossips to node 5.
Node 5 received gossip from node 2, neighborship list: [ 5 4 11 4 3 1 ]
Node 4 gossips to node 0.
Node 5 gossips to node 0.


Node 5 failed.

Terminating...



A sample output of the above example may look like this:
```
Node IDs:  [1 0]
Listener:1
Listener:0
GOSSIP msg sent by 1 to 0
GOSSIP msg received by 0 from 1
GOSSIP msg sent by 0 to 1
GOSSIP msg received by 1 from 0
GOSSIP msg sent by 1 to 0
Terminating 0 ...
GOSSIP msg received by 0 from 1
GOSSIP msg sent by 1 to 0
GOSSIP msg sent by 1 to 0
GOSSIP msg sent by 1 to 0
GOSSIP msg sent by 1 to 0
GOSSIP msg sent by 1 to 0
Terminating 1 ...
```

## A Gossip Style Failure Detection Service

In this homework, you are asked to modify and extend the above example to implement a gossip style failure detection protocol based on the following publication: https://www.cs.cornell.edu/home/rvr/papers/GossipFD.pdf. The protocol was also covered in the lecture. The essential points are summarized below (also see Section 2 in the paper and revisit the slides of Lecture 4 -- Coordination). 

Each node maintains a list with for each known node its address and an integer which is going to be used for failure detection. We call the integer the _heartbeat counter_. Every $T_{gossip}$ seconds, each node increments its own heartbeat counter, and selects one other node at random to send its list to. Upon receipt of such a gossip message, a node merges the list in the message with its own list, and adopts the maximum heartbeat counter for each node.

Each node also maintains, for each other node in the list, the last time that its corresponding heartbeat counter has increased. If the heartbeat counter has not increased for more than $T_{fail}$ seconds, then the node is considered failed. $T_{fail}$ is selected so that the probability that anybody makes an erroneous failure detection is less than some small threshold $P_{mistake}$.

After a node is considered faulty, it cannot immediately be forgotten about. The problem is that not all nodes will detect failures at the same time, and thus a node A may receive a gossip about another node B that A has previously detected as faulty. If A had forgotten about B, it would reinstall B in its membership list, since A would think that it was seeing B’s heartbeat for the first time. A would continue to gossip this information on to other nodes, and, in effect, the faulty node B never quite disappears from the membership.

Therefore, the failure detector does not remove a node from its membership list until after $T_{cleanup}$ seconds ($T_{cleanup} \geq T_{fail}$). $T_{cleanup}$ is chosen so that the probability that a gossip is received about this node, after it has been detected as faulty, is less than some small threshold $P_{cleanup}$. We can make $P_{cleanup}$ equal to $P_{fail}$ by setting $T_{cleanup}$ to $2 \times T_{fail}$.

To see why, let B be some failed node, and consider another node A that heard B’s last heartbeat at time $t$. With probability $P_{fail}$, every other node will have heard B’s last heartbeat by $t+T_{fail}$, and so every process will fail B by time $t+2\times T_{fail}$. Thus, if we set $T_{cleanup}$ to $2\times T_{fail}$, then with probability $P_{fail}$, no failed node will ever reappear on A’s list of nodes once that node has been removed.

## 1 - Gossiping Neighborship Lists [6 points]

**Your task:** Modify the sample code above to make nodes gossip their neighborship lists to a randomly chosen neighbor.

## 2 - Receiving and Updating Neighborship Lists [6 points]

**Your task:** Extend the code to correctly update the neighborship lists maintained by every node.

## 3 - Detect Node Failures [8 points]

**Your task:** Extend the code to correctly handle node failures by maintaining $T_{fail}$ and $T_{cleanup}$ timeouts and updating the neighborbood list according to the protocol.

## 4 - How to Submit Your Solution?

Download your notebook (File --> Download --> Download .ipynb) and send per email to [saukh@tugraz.at](mailto:saukh@tugraz.at).